In [1]:
# python 3
# -*- coding: utf-8 -*-
# load basics library
import pandas as pd
import numpy as np
import calendar
from sklearn.cluster import MiniBatchKMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
# DL
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers, callbacks, optimizers
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

#import commond.ipynb from same folder
import import_ipynb
from common import *

Using TensorFlow backend.


importing Jupyter notebook from common.ipynb


In [2]:
trainX, trainy, testX, testy = load_dataSet()
print("trainX shape: ", trainX.shape)
print("trainy shape: ", trainy.shape)
print("testX  shape: ", testX.shape)
print("testy  shape: ", testy.shape)
trainX, scaler_trainX = preprocess_data(trainX.values)
trainy = trainy.values
testX, scaler_testX = preprocess_data(testX.values)
testy = testy.values


print("trainX shape: ", trainX.shape)
print("trainy shape: ", trainy.shape)
print("testX  shape: ", testX.shape)
print("testy  shape: ", testy.shape)

   VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2017-01-07 12:47:25   2017-01-07 13:07:19                  N   
1         2  2017-01-21 12:21:04   2017-01-21 12:22:57                  N   

   RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0           1            40           141              1.0           2.91   
1           1            74            74              1.0           0.36   

   fare_amount ...   ehail_fee  improvement_surcharge  total_amount  \
0         13.0 ...         NaN                    0.3         16.56   
1          3.5 ...         NaN                    0.3          4.30   

   payment_type  trip_type  duration  day_of_week  weekend      speed  tip  
0             1        1.0    1194.0            5        1   8.773869    1  
1             2        1.0     113.0            5        1  11.469027    0  

[2 rows x 24 columns]
trainX shape:  (2086637, 15)
trainy shape:  (2086637,)
testX  shape:

In [3]:
# create model
model = Sequential()
model.add(Dense(1024, input_dim=trainX.shape[1], kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.71))
model.add(Dense(512, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(256, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(1, activation='linear'))

# Compile model
#model.compile(loss='mean_squared_error', optimizer=SGD(lr=0.02, momentum=0.9), metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])



In [4]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              16384     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total para

In [5]:
# simple early stopping, With GPU enabled
es = EarlyStopping(monitor='loss', mode='min', verbose=1)

XTraining, XValidation, YTraining, YValidation = train_test_split(trainX, trainy, test_size=0.1, shuffle= True) # before model building

#Use a Manual Verification Dataset
history = model.fit(XTraining, YTraining, validation_data=(XValidation, YValidation),
                    epochs=100, batch_size=32, shuffle=True, callbacks=[es])

print ('='*70)
print ('TRAIN RESULT : ')
print (history.history)
print ('='*70)

Train on 1877973 samples, validate on 208664 samples
Epoch 1/100
1877973/1877973 [==============================] - 150s 80us/step - loss: 0.3621 - acc: 0.6322 - val_loss: 0.1133 - val_acc: 0.6488
Epoch 2/100
1877973/1877973 [==============================] - 140s 74us/step - loss: 0.3629 - acc: 0.6354 - val_loss: 0.1688 - val_acc: 0.6496
Epoch 00002: early stopping
TRAIN RESULT : 
{'val_loss': [0.11325288093486031, 0.16877075914711004], 'val_acc': [0.6487702718268452, 0.6496089406916231], 'loss': [0.3621398318490818, 0.36290341976388557], 'acc': [0.6321794828789318, 0.6353978465081643]}


＃#　Below is not using GPU
![noGPU.png](picture/noGPU.png)